In [1]:
import neo4j
import pandas as pd
from utils_neo4j import init, add_station_relationships
import pandas as pd

In [2]:
NEO4J_URI = "bolt://localhost:7689"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "busticket123"

driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
init(driver)

In [3]:
# load departures preproc from json
departures_preproc = pd.read_csv("data_geofox/departures_all_preproc2.csv")

In [4]:
departures_preproc

,from_station,from_station_id,to_station,line_unique,line_name,line_info,departure_times,arrival_times,duration
0,A Burgwedel,Master:86007,Basselweg,HHA-B:603_HHA-B#Grothwisch#Rathausmarkt,603,Niederflur-Nachtbus,['2025-04-03 01:02:00'],['2025-04-03 01:03:00'],1
1,A Burgwedel,Master:86007,Bezirksamt Eimsbüttel,HHA-B:5_HHA-B#A Burgwedel#Hauptbahnhof/ZOB,5,Niederflur Metrobus,"['2025-04-03 05:04:00', '2025-04-03 19:34:00',...","['2025-04-03 05:05:00', '2025-04-03 19:35:00',...",1
2,A Burgwedel,Master:86007,Bf. Dammtor,HHA-B:5_HHA-B#A Burgwedel#Hauptbahnhof/ZOB,5,Niederflur Metrobus,"['2025-04-03 06:49:00', '2025-04-03 06:59:00',...","['2025-04-03 06:50:00', '2025-04-03 07:00:00',...",1
3,A Burgwedel,Master:86007,Bindfeldweg,HHA-B:5_HHA-B#A Burgwedel#Hauptbahnhof/ZOB,5,Niederflur Metrobus,"['2025-04-03 05:53:00', '2025-04-03 06:03:00',...","['2025-04-03 05:54:00', '2025-04-03 06:04:00',...",1
4,A Burgwedel,Master:86007,Freizeitbad Bondenwald,HHA-B:5_HHA-B#A Burgwedel#Hauptbahnhof/ZOB,5,Niederflur Metrobus,"['2025-04-03 07:05:00', '2025-04-03 07:15:00',...","['2025-04-03 07:06:00', '2025-04-03 07:16:00',...",1
...,...,...,...,...,...,...,...,...,...
65318,Überseering (West),Master:70017,Ohlsdorfer Straße (Planetarium),HHA-B:20_HHA-B#S Rübenkamp#Winterhuder Marktplatz,20,Niederflur Metrobus,"['2025-04-03 23:19:00', '2025-04-03 23:39:00']","['2025-04-03 23:21:00', '2025-04-03 23:41:00']",2
65319,Überseering (West),Master:70017,S Poppenbüttel (Wentzelplatz),HHA-B:179_HHA-B#S Poppenbüttel (Wentzelplatz)#...,179,Niederflur Stadtbus,"['2025-04-03 07:50:00', '2025-04-03 08:10:00',...","['2025-04-03 07:51:00', '2025-04-03 08:11:00',...",2
65320,Überseering (West),Master:70017,Schulbergredder,HHA-B:179_HHA-B#S Poppenbüttel (Wentzelplatz)#...,179,Niederflur Stadtbus,['2025-04-03 05:27:00'],['2025-04-03 05:29:00'],2
65321,Überseering (West),Master:70017,Schulweg,HHA-B:20_HHA-B#Bf. Altona#S Rübenkamp,20,Niederflur Metrobus,"['2025-04-03 05:13:00', '2025-04-03 05:33:00',...","['2025-04-03 05:14:00', '2025-04-03 05:34:00',...",1


In [5]:
processed_data = departures_preproc.to_dict('records')

In [26]:
# Execute the query
with driver.session() as session:
    add_station_relationships(processed_data, session)

driver.close()

Adding 636528 relationships to Neo4j...


Adding relationships to Neo4j:   1%|          | 5/637 [00:08<17:01,  1.62s/it]

Skipping departure from 'Borgfelder Straße' to 'U Hammer Kirche (Hammer Landstraße)' - Destination station does not exist
Skipping departure from 'U Burgstraße' to 'U Hammer Kirche (Hammer Landstraße)' - Destination station does not exist
Skipping departure from 'U Hammer Kirche' to 'U Hammer Kirche (Hammer Landstraße)' - Destination station does not exist
Skipping departure from 'U Hammer Kirche' to 'U Hammer Kirche (Hammer Landstraße)' - Destination station does not exist
Skipping departure from 'U Hammer Kirche' to 'U Hammer Kirche (Hammer Landstraße)' - Destination station does not exist
Skipping departure from 'U Hammer Kirche' to 'U Hammer Kirche (Hammer Landstraße)' - Destination station does not exist
Skipping departure from 'U Hammer Kirche' to 'U Hammer Kirche (Hammer Landstraße)' - Destination station does not exist
Skipping departure from 'U Hammer Kirche' to 'U Hammer Kirche (Hammer Landstraße)' - Destination station does not exist
Skipping departure from 'U Hammer Kirche'

Adding relationships to Neo4j:   1%|          | 6/637 [00:10<16:36,  1.58s/it]

Skipping departure from 'U Hammer Kirche' to 'U Hammer Kirche (Hammer Landstraße)' - Destination station does not exist
Skipping departure from 'U Hammer Kirche' to 'U Hammer Kirche (Hammer Landstraße)' - Destination station does not exist
Skipping departure from 'U Hammer Kirche' to 'U Hammer Kirche (Hammer Landstraße)' - Destination station does not exist
Skipping departure from 'U Hammer Kirche' to 'U Hammer Kirche (Hammer Landstraße)' - Destination station does not exist
Skipping departure from 'U Hammer Kirche' to 'U Hammer Kirche (Hammer Landstraße)' - Destination station does not exist
Skipping departure from 'U Hammer Kirche' to 'U Hammer Kirche (Hammer Landstraße)' - Destination station does not exist
Skipping departure from 'U Hammer Kirche' to 'U Hammer Kirche (Hammer Landstraße)' - Destination station does not exist
Skipping departure from 'U Hammer Kirche' to 'U Hammer Kirche (Hammer Landstraße)' - Destination station does not exist
Skipping departure from 'U Hammer Kirche

...

Skipping departure from 'Hamburg-Altona' to 'Horst (Holst)' - Destination station does not exist
Skipping departure from 'Hamburg-Altona' to 'Horst (Holst)' - Destination station does not exist
Skipping departure from 'Hamburg-Altona' to 'Horst (Holst)' - Destination station does not exist
Skipping departure from 'Hamburg-Altona' to 'Horst (Holst)' - Destination station does not exist


Adding relationships to Neo4j:  99%|█████████▊| 628/637 [15:58<00:13,  1.53s/it]

Skipping departure from 'Hamburg-Altona' to 'Horst (Holst)' - Destination station does not exist
Skipping departure from 'Hamburg-Altona' to 'Horst (Holst)' - Destination station does not exist


Adding relationships to Neo4j: 100%|██████████| 637/637 [16:11<00:00,  1.53s/it]

  - 227 due to missing destination stations
Creating additional indexes for optimized querying...
Relationship creation complete!
